In [ ]:
import os
import subprocess
import time
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

# Google API configuration
SCOPES = ["https://www.googleapis.com/auth/youtube.upload"]
CLIENT_SECRETS_FILE = "client_secret.json"
TOKEN_FILE = "token.json"
YOUTUBE_MAX_DURATION = 43200  # 12 hours in seconds

MERGE_TIMEOUT = 1200  # 20 minutes
CONVERT_TIMEOUT = 7200  # 2 hours

def get_authenticated_service():
    """
    Authenticate with YouTube API and return a service object.
    Reuses saved credentials to avoid repeated logins.
    """
    creds = None

    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            try:
                creds.refresh(Request())
            except:
                creds = None  # Force re-authentication if refresh fails

        if creds is None:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
            creds = flow.run_local_server(port=0)

        with open(TOKEN_FILE, "w") as token_file:
            token_file.write(creds.to_json())

    return build("youtube", "v3", credentials=creds)

def get_video_duration(video_path):
    """
    Returns the duration of a video in seconds.
    """
    try:
        result = subprocess.run(
            ["ffprobe", "-v", "error", "-show_entries", "format=duration",
             "-of", "default=noprint_wrappers=1:nokey=1", video_path],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
        )
        return float(result.stdout.strip())
    except Exception as e:
        print(f"⚠️ Warning: Could not read duration for {video_path}. Error: {e}")
        return None

def try_merge_videos(folder_path, video_files, expected_duration, merge_attempt):
    """
    Attempts to merge videos directly without conversion.
    If the merged duration is incorrect, fall back to conversion.
    """
    file_list_path = os.path.join(folder_path, "file_list.txt")
    merged_video_path = os.path.join(folder_path, "merged_video.mp4")

    with open(file_list_path, "w", encoding="utf-8") as f:
        for video_file in video_files:
            f.write(f"file '{video_file}'\n")

    print(f"🔗 Attempt {merge_attempt}: Merging videos... (Timeout: {MERGE_TIMEOUT // 60} min)")

    try:
        subprocess.run(
            ["ffmpeg", "-f", "concat", "-safe", "0", "-i", file_list_path, "-c", "copy", merged_video_path],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, timeout=MERGE_TIMEOUT
        )
    except subprocess.TimeoutExpired:
        print(f"⏳ Merge attempt {merge_attempt} **timed out** after {MERGE_TIMEOUT // 60} minutes! Skipping...")
        return None  # Skip to conversion if merge times out

    merged_duration = get_video_duration(merged_video_path)
    if merged_duration and abs(merged_duration - expected_duration) <= 5:
        print(f"✅ Merge successful! Duration: {merged_duration:.2f} sec")
        return merged_video_path
    else:
        print(f"❌ Merge failed. Merged duration: {merged_duration:.2f} sec, Expected: {expected_duration:.2f} sec")
        os.remove(merged_video_path)  # Delete incorrect merge
        return None  # Signal to perform full conversion

def convert_videos(folder_path, video_files):
    """
    Converts videos to a uniform format.
    """
    converted_folder = os.path.join(folder_path, "converted_videos")
    os.makedirs(converted_folder, exist_ok=True)
    converted_files = []

    print(f"🔄 Starting video conversion... (Timeout: {CONVERT_TIMEOUT // 3600} hours)")

    start_time = time.time()
    for video in video_files:
        converted_video = os.path.join(converted_folder, os.path.basename(video))
        
        try:
            subprocess.run(
                ["ffmpeg", "-i", video, "-c:v", "libx264", "-c:a", "aac", "-strict", "experimental", converted_video],
                stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, timeout=CONVERT_TIMEOUT
            )
        except subprocess.TimeoutExpired:
            print(f"⏳ Conversion **timed out** after {CONVERT_TIMEOUT // 3600} hours! Aborting this subfolder...")
            return []  # Return an empty list to indicate failure

        converted_files.append(converted_video)

    print("✅ Video conversion complete!")
    return converted_files

def merge_videos_and_create_timestamps(folder_path, video_files, part_number=None):
    """
    Merges videos, creates timestamps, and returns the final video path.
    """
    print(f"📂 Checking folder: {folder_path}")

    timestamps = []
    total_duration = sum(get_video_duration(f) or 0 for f in video_files)

    merged_video_name = f"merged_video_Part_{part_number}.mp4" if part_number else "merged_video.mp4"
    merged_video_path = os.path.join(folder_path, merged_video_name)

    merged_video_path = try_merge_videos(folder_path, video_files, total_duration, merge_attempt=1)
    if merged_video_path:
        return merged_video_path, "\n".join(f"{os.path.basename(f)}" for f in video_files), total_duration

    print("🔄 Converting videos due to merge failure...")
    converted_files = convert_videos(folder_path, video_files)

    if not converted_files:
        print(f"❌ Conversion failed for {folder_path}. Skipping upload.")
        return None, None, None

    return try_merge_videos(folder_path, converted_files, total_duration, merge_attempt=2), "\n".join(f"{os.path.basename(f)}" for f in converted_files), total_duration

import ssl
import time

def upload_to_youtube(video_path, title, description, folder_path, max_retries=3, retry_delay=60):
    """
    Uploads a video to YouTube with automatic retry on SSL failures.
    """
    print(f"📤 Uploading: {title}...")

    youtube = get_authenticated_service()
    request = youtube.videos().insert(
        part="snippet,status",
        body={
            "snippet": {
                "title": title,
                "description": description,
                "tags": ["training", "tutorial", "video"],
                "categoryId": "27"
            },
            "status": {
                "privacyStatus": "unlisted"
            }
        },
        media_body=MediaFileUpload(video_path, chunksize=-1, resumable=True)
    )

    response = None
    attempt = 0

    while response is None:
        try:
            status, response = request.next_chunk()
            if status:
                print(f"Uploaded {int(status.progress() * 100)}%")
        except ssl.SSLEOFError as e:
            attempt += 1
            if attempt > max_retries:
                print(f"❌ Upload failed after {max_retries} attempts due to SSL error.")
                raise Exception(f"SSL Upload Error: {e}")
            
            print(f"🔄 SSL error detected! Retrying in {retry_delay} seconds... (Attempt {attempt}/{max_retries})")
            time.sleep(retry_delay)  # Wait before retrying
        except Exception as e:
            raise Exception(f"Error during upload: {e}")

    video_url = f"https://www.youtube.com/watch?v={response['id']}"
    with open(os.path.join(folder_path, "youtube_link.txt"), "w") as f:
        f.write(video_url + "\n")

    print(f"✅ Upload complete! Video URL: {video_url}")
    return video_url


if __name__ == "__main__":
    parent_folder = input("📂 Enter parent folder: ").strip()

    for subfolder in sorted(os.listdir(parent_folder)):
        subfolder_path = os.path.join(parent_folder, subfolder)

        if os.path.isdir(subfolder_path) and any(f.lower().endswith('.mp4') for f in os.listdir(subfolder_path)):
            print(f"🚀 Processing folder: {subfolder_path}")

            video_files = sorted([os.path.join(subfolder_path, f) for f in os.listdir(subfolder_path) if f.lower().endswith('.mp4')])
            total_duration = sum(get_video_duration(f) or 0 for f in video_files)

            merged_video_path, description, _ = merge_videos_and_create_timestamps(subfolder_path, video_files)
            if merged_video_path:
                upload_to_youtube(merged_video_path, subfolder, description, subfolder_path)

print("✅ All subfolders processed!")
